In [ ]:
import datetime as dt
import math
import os

import holoviews as hv
import keras_metrics as km
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from influxdb import DataFrameClient
from keras import Sequential
from keras import backend as K
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Flatten,
                          Input, RepeatVector, TimeDistributed)
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import Concatenate, concatenate
from keras.models import Model
from numpy.random import seed
from pylab import rcParams
from scipy import stats
from sklearn.externals import joblib
from sklearn.metrics import (auc, classification_report, confusion_matrix,
                             f1_score, precision_recall_curve,
                             precision_recall_fscore_support, recall_score,
                             roc_curve)
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import class_weight
from tensorflow import set_random_seed

import ricercando as ric

database_ip = '46.101.250.119'
ric.set_connection_params(host=database_ip)
cli = DataFrameClient(database_ip, 8086, 'monroe', 'secure', 'monroe')
cli.switch_database('monroe')

seed(7)
set_random_seed(11)
rcParams['figure.figsize'] = 8, 6
LABELS = ["False","True"]

In [ ]:
# Select nodes
train_nodes = [
    {
        "node_id": '601',
        "ICCID": '89390100001965067610',
        "start_time": '2018-01-01',
        "end_time": '2018-01-30'
    },
    {
        "node_id": '608',
        "ICCID": '8946071512360089522',
        "start_time": '2018-01-01',
        "end_time": '2018-01-30'
    },
    {
        "node_id": '609',
        "ICCID": '89460850007007786482',
        "start_time": '2018-01-01',
        "end_time": '2018-01-30'
    },
    {
        "node_id": '610',
        "ICCID": '8939104160000392272',
        "start_time": '2018-01-01',
        "end_time": '2018-01-30'
    },
    {
        "node_id": '612',
        "ICCID": '8939104160000392231',
        "start_time": '2018-01-01',
        "end_time": '2018-01-29'
    },
    {
        "node_id": '613',
        "ICCID": '89390100001965068626',
        "start_time": '2018-01-01',
        "end_time": '2018-01-29'
    }
    
]

In [ ]:
# Preprocess input

n_features = 1
lookback = 240
window_sizes = [1,5,10,20,40]

X_scaled_windows = []

# create input for every window
for window_size in window_sizes:
    X_array = []
    y_array = []
    
    kfold_array_train = []
    kfold_array_test = []
    
    index_for_cv = 0
    
    num_splits = 5
    for i in range(num_splits):
            kfold_array_train.append([])
            kfold_array_test.append([])

    for node in train_nodes:
        node_id = node["node_id"]
        ICCID = node["ICCID"]
        start_time = node["start_time"]
        end_time = node["end_time"]

        datasets = cli.query("select * from class_1m where NodeId='{}' and time >= '{}' and time <= '{}' ".format(node_id,start_time,end_time))
        df = ric.getdf(tables="ping", nodeid=node_id,  start_time= start_time, end_time=end_time, freq="1m")
        df = df[df['Iccid'] == ICCID]

        # merge together class and df
        class_feature = datasets['class_1m'].copy()

        class_feature = class_feature.drop(columns=['NodeId'])
        class_feature.index = class_feature.index.tz_localize(None)
        class_feature['time'] = class_feature.index
        df['time'] = df.index
        df.index.name = None
        df = pd.merge(df, class_feature,  how='inner', left_on=['Iccid','time'], right_on = ['Iccid','time'])
        df.index = df['time']
        df = df.drop(columns=['time'])
        df.index.name = 'time'
        df_analise = df.copy()

        # delay it for lookback value and predict from last element

        df = df_analise.copy()
        df = df.dropna(subset=['RTT'])
        df.index = list(range(len(df.index)))
        df = df[["RTT","Class"]]
        df['Class'] = df['Class'].values * 1

        df = df.fillna(0)      
        df['RTT'] = df['RTT'].rolling(window_size, min_periods=1).mean()
        
        # create padding for empty values
        
        first_RTT = df['RTT'][0]
        last_RTT = df['RTT'].values[-1]

        for i in range((int(lookback/2))-1,-1,-1):
            df['RTT_-{}'.format(i)] = df['RTT'].shift(periods=i).fillna(first_RTT)

        for i in range(1,(int(lookback/2)+1),1):
            df['RTT_{}'.format(i)] = df['RTT'].shift(periods=-i).fillna(last_RTT)

        columns_list = list(df.columns.values)
        features_names = list(filter(lambda x : "RTT_" in x, columns_list))


        X = df[features_names].values
        y = df["Class"].values
        
        X_array.append(X)
        y_array.append(y)

        kfold = KFold(n_splits=num_splits, shuffle=False)
        
        number_of_rows = X.shape[0]
        print(number_of_rows)
        
        # create fold indexes        
        l = 0
        for train, test in kfold.split(X, y):
            
            mod_train = list(map(lambda x: x + index_for_cv, train))
            mod_test = list(map(lambda x: x + index_for_cv, test))
            
            kfold_array_train[l] += mod_train
            kfold_array_test[l] += mod_test
            
            l += 1
            
        index_for_cv += number_of_rows

    X = np.concatenate(X_array)
    y = np.concatenate(y_array)

    sc = MinMaxScaler()    
    X_scaled = sc.fit_transform(X) 
    X_scaled = X_scaled.reshape(X.shape[0], lookback, n_features)
    X_scaled_windows.append(X_scaled)

    
# final input is X_scaled_windows


In [ ]:
# define custom scoring

# combining intervals
def customise_score(y_testt, y_predd, offset = 5, mark_as=1):
    
    
    y_t = np.copy(y_testt)
    y_p = np.copy(y_predd)

    
    # Fill-in gaps betwwen test
    for i in range(len(y_t)):
        if y_t[i] and any(y_t[i+1:i+offset+1]):
            for j in range(1,offset+1):
                if y_t[i+j]:
                    break
                else:
                    y_t[i+j] = mark_as
        
    # Fill-in gaps betwwen pred
    for i in range(len(y_p)):
        if y_p[i] and any(y_p[i+1:i+offset+1]):
            for j in range(1,offset+1):
                if y_p[i+j]:
                    break
                else:
                    y_p[i+j] = mark_as
                
    return y_t, y_p

# counting intervals

def customise_score_for_readable(y_testt, y_predd, offset = 8, offset_pred = 8, mark_as = 1, mark_as_inverse = 0):
    
    
    y_t = np.copy(y_testt)
    y_p = np.copy(y_predd)


    # Fill-in gaps between test marked True Classes
    for i in range(len(y_t)):
        if y_t[i] and any(y_t[i+1:i+offset+1]):
            for j in range(1,offset+1):
                if y_t[i+j]:
                    break
                else:
                    y_t[i+j] = mark_as
                    
                    
    # Fill-in gaps between pred marked True Classes
    for i in range(len(y_p)):
        if y_p[i] and any(y_p[i+1:i+offset_pred+1]):
            for j in range(1,offset_pred+1):
                if y_p[i+j]:
                    break
                else:
                    y_p[i+j] = mark_as
                
    return y_t, y_p
            

def customise_score_readable(*args, **kwargs):
   
    y_t, y_p = customise_score_for_readable(*args, **kwargs)
    
    if len(y_t) != len(y_p):
        raise Exception("Invalid length od y_p and y_t, should be same")
    
    i = 0
    
    new_y_t = []
    new_y_p = []
    
    
    num_TN = 1
    
    # find TP and Fn
    
    while i < len(y_t):
        if y_t[i]:
            j = 1
            while y_t[i+j]:
                j += 1
            
            if any(y_p[i:i+j]):
                new_y_t.append(1)
                new_y_p.append(1)
                num_TN += 1
                i = i + j
                continue
            else:
                new_y_t.append(1)
                new_y_p.append(0)
                i = i + j
                
        i += 1
    
    # find TN - they dont matter- but number same as number of anomaly zones
    
    for i in range(num_TN):
        new_y_t.append(0)
        new_y_p.append(0)
    
    
    # find FP
                
    while i < len(y_p):
        if y_p[i]:
            j = 1
            while y_p[i+j]:
                j += 1
            
            if not any(y_t[i:i+j]):
                new_y_t.append(0)
                new_y_p.append(1)
                i = i + j
                continue
            else:
                i = i + j
                continue
        i += 1
        
    
    return new_y_t, new_y_p

In [ ]:
# define keras model

def getModel():
    lr = 0.0001

    input_branches = []
    output_branches = []

    for i in range(len(window_sizes)):
        visible = Input(shape=(lookback,n_features))
        conv1 = Conv1D(filters=16, kernel_size=3, activation='relu')(visible)
        pool1 = MaxPooling1D(pool_size=2)(conv1)

        input_branches.append(visible)
        output_branches.append(pool1)
    merge = concatenate(output_branches)

    conv2 = Conv1D(filters=64, kernel_size=9, activation='relu')(merge)
    pool2 = MaxPooling1D(pool_size=4)(conv2)
    flat2 = Flatten()(pool2)

    hidden1 = Dense(256, activation='relu')(flat2)

    dropout1 = Dropout(0.4)(hidden1)

    hidden2 = Dense(128, activation='relu')(dropout1)

    norm2 = Dropout(0.4)(hidden2)

    output = Dense(1, activation='sigmoid')(norm2)
    model = Model(inputs=input_branches, outputs=output)

    adam = optimizers.Adam(lr)
    model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy',km.precision(), km.recall()])

    return model

In [ ]:
runda = 1

for indexindex in range(num_splits):
    
    train = kfold_array_train[indexindex]
    test = kfold_array_test[indexindex]
    
    X_train_win_arr = []
    X_valid_win_arr = []
    
    
    y_train = y[train]
    y_valid = y[test]
    
    
    for X_window in X_scaled_windows:
        
        X_train_win_arr.append(X_window[train])
        X_valid_win_arr.append(X_window[test])
        

    model = getModel()
    
    es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    
    class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

    history = model.fit(X_train_win_arr, y_train, batch_size=128, epochs=20, verbose=2, validation_data=(X_valid_win_arr,y_valid), class_weight=class_weights, callbacks= [es])
    
    # show history
    print(history.history.keys())
    print("----------------------------")
    print("številka folda",runda)
    
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['precision'])
    plt.plot(history.history['val_precision'])
    plt.title('Model precision')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['recall'])
    plt.plot(history.history['val_recall'])
    plt.title('Model recall')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
    y_test = y_valid
    X_test_scaled_windows = X_valid_win_arr

    y_true, y_pred = y_test, model.predict(X_test_scaled_windows)
    fpr, tpr, threshold = roc_curve(y_true, y_pred)

    results = model.evaluate(X_test_scaled_windows, y_true)
    print('test loss, test acc, test prec, test recall:', results)

    roc_auc = auc(fpr, tpr)
    print("BREZ PADDINGA")
    #no Padding
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()


    threshold_fixed = 0.5
    pred_y = [1 if e > threshold_fixed else 0 for e in y_pred]
    conf_matrix = confusion_matrix(y_true,pred_y)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
    plt.title("Matrika zamenjav")
    plt.ylabel('Označen razred')
    plt.xlabel('Napovedan razred')
    plt.show()

    print("classification report za območja brez paddinga")
    print(classification_report(y_true, pred_y))



    #with Padding
    print("S PADDINGOM")
    y_true, pred_y = customise_score(y_true, pred_y,10)
    fpr, tpr, threshold = roc_curve(y_true, pred_y)
    roc_auc = auc(fpr, tpr)
    conf_matrix = confusion_matrix(y_true,pred_y)


    plt.title('ROC - združevanje intervalov')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
    plt.title("Matrika zamenjav - združevanje intervalov")
    plt.ylabel('Označen razred')
    plt.xlabel('Napovedan razred')
    plt.show()

    print("classification report za območja padding")
    print(classification_report(y_true, pred_y))


    # counting intervals
    print("ŠTETJE INTERVALOV")

    y_true, y_pred = y_test, model.predict(X_test_scaled_windows)
    threshold_fixed = 0.5
    pred_y = [1 if e > threshold_fixed else 0 for e in y_pred]

    y_true, pred_y = customise_score_readable(y_true, pred_y, offset = 10, offset_pred = 10)
    fpr, tpr, threshold = roc_curve(y_true, pred_y)
    roc_auc = auc(fpr, tpr)
    conf_matrix = confusion_matrix(y_true,pred_y)


    plt.title('ROC - štetje intervalov')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
    plt.title("Matrika zamenjav - štetje območij anomalij")
    plt.ylabel('Označen razred')
    plt.xlabel('Napovedan razred')
    plt.show()

    print("classification report za območje štetja intervalov")
    print(classification_report(y_true, pred_y))
    
    runda += 1
    
    if indexindex != 4:
        K.clear_session()
    